# 訓練資料前處理

### 第一階段
- 去除髒話
- 翻譯

In [1]:
from better_profanity import profanity
from googletrans import Translator
import csv

### 引入翻譯以及髒話清理套件

profanity 可以篩選出許多常見英文髒話，也會偵測類似字 (EX : ass, 4ss, bitch, b1tch)

In [3]:
translator = Translator()
datalist = []

with open('abusive.csv', newline='', encoding='utf-8') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        #清除髒字
        text = profanity.censor(row, "")
        try:
            result = translator.translate(text, dest='zh-tw').text
            datalist.append(result)
        except:
            continue

### 存成 CSV
#### stage1.csv

In [4]:
import pandas as pd
bar_name = list(datalist) 
store = pd.DataFrame(data = datalist)
store.to_csv('abusive_clean_stage1.csv', encoding = 'utf-8_sig',index = True)

### 第二階段
- 清除其餘髒東西 (URL, RT, @...)
####  Regex 邏輯：

##### 照順序往下掃

- 將用戶 @blablabla 替換成 "你"
- 將網址去除
- 空格去除
- 非中文去除 (除了英文，還可以去除 emoji 和 # 之類的麻煩符號)

In [5]:
import re

user = re.compile('(@[^\w]*)')
url = re.compile('((https?):((//)|(\\\\))+[\w\d:#@%/;$()~_?\+-=\\\.&]*)')
spacechar = re.compile('(\\s){1,}')
non_chinese = re.compile('([^\u4e00-\u9fa5，。！：])')

### 開始清理
##### 將 stage1.csv 餵進去二次清洗

In [7]:
datalist = []

with open('abusive_clean_stage1.csv', newline='', encoding='utf-8') as csvfile:
  rows = csv.reader(csvfile)
  for row in rows:
        content = re.sub(url, "", row[1])
        content = re.sub(user, "你", content)
        content = re.sub("RT", "", content)
        content = re.sub(spacechar, "", content)
        content = re.sub(non_chinese, "", content)
        content = re.sub("你：", "", content)
        content = re.sub("白痴", "人", content)
        #去除所有疊字
        content = re.sub(r"(.)(\1+)", r"\1", content)
        if content != "":
            datalist.append(content)

內容
另一個人
正如南希裡根所說的那樣，只是說不！
但是他仍然和他一起開始吸煙和喝酒不好的組合可能看起來像
愚人節，如果您感覺不到這種感覺，那麼您的莫蒂斯：
無法獲得我的錢是我離開了
上帝，你是可悲的。
你擔心有人你
該死的院長只是讓入睡。這場比賽還表明了為什麼它在預秀上。無聊，就像他媽的
我討厭一個我在家裡忘了錢包，但我有我的身份證！
我只想哭得如此糟糕的看
。我不好你不能我
這是一個漫長的四月傻瓜，我祈禱有人說這是一個坏笑話
噁心。侮辱。公園不是慈善機構。給他們預算，。你
您不討厭在嘗試之前在雞巴上放鹽的人嗎
不要說謊，我們的朋友放了一張圖片，我們剛寫下面的照片
不要和我一起睡覺那就不要和我說話。而且永遠不要和我說不好的回合
格拉斯利是騙子為支持者說一切。所有可笑的
告訴我的人，是我，你的茶中的糖，你真的生氣了！
在上取消關注壞母狗只是為了跟隨一些吉利的亞洲人
你您仍然與那支球隊一起搖擺不定，生病了，我嘗試了他
耶穌，出去續集看起來很恐怖
不允許您更改電子郵件地址的系統您在做什麼您是由人建造的嗎
聖潔！你拉賽車卡上市長最大的屁股！得到他的辦公室！
我重複血腥發生了什麼！
我很討厭看到這條推文。
當我喜歡你時，我是一個討厭的怪胎
耶穌，出去續集看起來很恐怖
分鐘前開放是如此煩人。發送消息給的發送給他們以回复
他媽的清酒走開了
這些秘密討厭的原因
人稱它們為抗酸劑而不是加力者
我也病了這些頭
你是個人，因為我不在乎你在說什麼。
你，啤酒購買怪異的人認為特朗普將其洩露給指甲米飯嗎你在開玩笑嗎並犧牲弗林。泥是準則
！
你我會為你去蝙蝠！人們怎麼了！也
你他說他不會再在這裡待分鐘！
您知道信任病態騙子的人們會發生什麼他們過來了！反复！
警察抱著我。警察問官員，他回答。說握住人
，
我討厭他們
。
我們想念你的懶惰，杜爾，是
姐我病了。
我不知道該說些什麼！我有雞皮！
他媽的你死了：一個可怕的歷史
你。
討厭看面試和我聽到一百萬次
我在學校周圍走來走去，不喜歡我，只是坐在周圍坐在那裡討厭
法院和鄧肯今晚很痛苦
我討厭春夏的一件事是血腥的鳥在凌晨點喚醒我。
我的你不要死！
我搬到德克薩斯州。那裡有些不好。
有人告訴我他們不喜歡貓頭鷹敢於你的臉如何使聲音變成如此可怕的詞，
營銷與煩人之間有區別
我國的一些人認為，這個政權正在努力帶來統一的民法典。
剛和她的老闆丈夫一起多汁她
你

#### 確認一下資料有沒有清乾淨

In [ ]:
datalist

### 存成 CSV
#### stage2.csv

In [8]:
import pandas as pd
bar_name = list(datalist) 
store = pd.DataFrame(data = datalist)
store.to_csv('abusive_clean_stage2.csv', encoding = 'utf-8_sig',index = True)